# [Housing Prices Competition for Kaggle Learn Users](https://www.kaggle.com/c/home-data-for-ml-course)

## Импорт пакетов

In [1]:
import pandas as pd
import numpy as np

from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_absolute_error, r2_score

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from category_encoders import OrdinalEncoder
from category_encoders import OneHotEncoder
from sklearn.compose import ColumnTransformer

## Загрузка данных

In [2]:
hpc_train = pd.read_csv("./../../../data/hpc_train.csv")
hpc_train

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [3]:
hpc_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [4]:
y = hpc_train.SalePrice
y

0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1460, dtype: int64

In [5]:
hpc_train.drop(columns=['SalePrice', 'Id'], inplace=True)
hpc_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal
1456,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal
1457,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal
1458,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal


In [6]:
mapping_ex_gd_ta_fa_po_no = {'Ex': 5, 
                             'Gd': 4, 
                             'TA': 3, 
                             'Fa': 2, 
                             'Po': 1, 
                             np.nan: 0}
#                              'NA': 0}
columns_ordered_ex_gd_ta_fa_po_no = ['BsmtQual', 
                                     'BsmtCond', 
                                     'FireplaceQu',
                                     'GarageCond']


mapping_gd_av_mn_no_na = {'Gd': 4, 
                          'Av': 3, 
                          'Mn': 2, 
                          'No': 1, 
                          np.nan: 0}
#                           'NA': 0}
columns_ordered_gd_av_mn_no_na = ['BsmtExposure']


mapping_glq_alq_blq_rec_lwq_unf_na  = {'GLQ': 6,
                                       'ALQ': 5,
                                       'BLQ': 4, 
                                       'Rec': 3 , 
                                       'LwQ': 2, 
                                       'Unf': 1, 
                                       np.nan: 0}
#                                        'NA': 0}
columns_ordered_glq_alq_blq_rec_lwq_unf_na = ['BsmtFinType1', 
                                              'BsmtFinType2']

mapping_electrical = {'SBrkr': 4, 
                      'FuseA': 3, 
                      'FuseF': 2, 
                      'FuseP': 1, 
                      'Mix': 0}
columns_ordered_electrical = ['Electrical']

mapping_ex_gd_ta_fa_no = {'Ex': 4, 
                          'Gd': 3, 
                          'TA': 2, 
                          'Fa': 1, 
                          np.nan: 0}
#                           'NA': 0}
columns_ordered_ex_gd_ta_fa_no = ['PoolQC']

mapping_gdprv_mnprv_gdwo_mnww_na = {'GdPrv': 4, 
                                    'MnPrv': 3, 
                                    'GdWo': 2, 
                                    'MnWw': 1, 
                                    np.nan: 0}
#                                     'NA': 0}
columns_ordered_gdprv_mnprv_gdwo_mnww_na = ['Fence']

ord_mapping = [
    {'col': 'BsmtQual', 'mapping': mapping_ex_gd_ta_fa_po_no},
    {'col': 'BsmtCond', 'mapping': mapping_ex_gd_ta_fa_po_no}, 
    {'col': 'FireplaceQu', 'mapping': mapping_ex_gd_ta_fa_po_no},
    {'col': 'GarageCond', 'mapping': mapping_ex_gd_ta_fa_po_no},
    {'col': 'BsmtExposure', 'mapping': mapping_gd_av_mn_no_na},
    {'col': 'BsmtFinType1', 'mapping': mapping_glq_alq_blq_rec_lwq_unf_na},
    {'col': 'BsmtFinType2', 'mapping': mapping_glq_alq_blq_rec_lwq_unf_na},
    {'col': 'Electrical', 'mapping': mapping_electrical},
    {'col': 'PoolQC', 'mapping': mapping_ex_gd_ta_fa_no},
    {'col': 'Fence', 'mapping': mapping_gdprv_mnprv_gdwo_mnww_na}
]

ord_encoder = OrdinalEncoder(mapping=ord_mapping, 
                                 return_df=True)
df = ord_encoder.fit_transform(hpc_train)
df[columns_ordered_ex_gd_ta_fa_po_no]

,BsmtQual,BsmtCond,FireplaceQu,GarageCond
0,4,3,0,3
1,4,3,3,3
2,4,3,3,3
3,3,4,4,3
4,4,3,3,3
...,...,...,...,...
1455,4,3,3,3
1456,4,3,3,3
1457,3,4,4,3
1458,3,3,0,3


In [15]:
s_imputer_none = SimpleImputer(missing_values=np.nan, 
                               strategy='constant', 
                               fill_value='None')
df_with_nan = hpc_train[['Alley', 
                         'MasVnrType',
                         'GarageType', 
                         'MiscFeature']]
df_with_nan

,Alley,MasVnrType,GarageType,MiscFeature
0,NaN,BrkFace,Attchd,NaN
1,NaN,None,Attchd,NaN
2,NaN,BrkFace,Attchd,NaN
3,NaN,None,Detchd,NaN
4,NaN,BrkFace,Attchd,NaN
...,...,...,...,...
1455,NaN,None,Attchd,NaN
1456,NaN,Stone,Attchd,NaN
1457,NaN,None,Attchd,Shed
1458,NaN,None,Attchd,NaN


In [17]:
s_imputer_none = s_imputer_none.fit(df_with_nan)

imputed_hpc_train = pd.DataFrame(s_imputer_none.transform(df_with_nan))
imputed_hpc_train.columns = ['Alley', 'MasVnrType', 'GarageType', 'MiscFeature']
imputed_hpc_train

,Alley,MasVnrType,GarageType,MiscFeature
0,None,BrkFace,Attchd,None
1,None,None,Attchd,None
2,None,BrkFace,Attchd,None
3,None,None,Detchd,None
4,None,BrkFace,Attchd,None
...,...,...,...,...
1455,None,None,Attchd,None
1456,None,Stone,Attchd,None
1457,None,None,Attchd,Shed
1458,None,None,Attchd,None


In [ ]:
oh_encoder = One(cols=['Alley', 'MasVnrType', 'GarageType', 'MiscFeature'], 
                 use_cat_names=True, 
                 return_df=True)
df = oh_encoder.fit_transform(hpc_train)